Cleaning Data

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import itertools

from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup

import calendar
from datetime import datetime, date, timedelta
import warnings
warnings.filterwarnings('ignore')

In [7]:
#Import Data by connecting to storms Database With SQLalchemy
Tornado_local = create_engine('postgresql://localhost:5432/storms')

In [8]:
sql_query = 'SELECT * FROM tornadoes2018 LIMIT 3' 
pd.read_sql(sql_query, Tornado_local)

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,200608,27,1942,200608,27,1946,1219535.0,5528248,TEXAS,48.0,...,2.0,SSW,FRITCH,35.63,-101.62,35.60,-101.62,Severe thunderstorms produced hail...damaging ...,The Emergency Manager from Hutchinson county o...,PDS
1,200608,24,1818,200608,24,1840,1219338.0,5528107,SOUTH DAKOTA,46.0,...,10.0,SE,HURON,44.32,-98.17,44.25,-98.07,None,A tornado severely damaged 3 houses and destro...,PDS
2,200608,24,1856,200608,24,1857,1219340.0,5528109,SOUTH DAKOTA,46.0,...,6.0,SSE,IROQUOIS,44.28,-97.80,44.28,-97.80,None,A brief tornado caused no reported damage.,PDS


In [11]:
sql_query = """
SELECT COUNT("TOR_F_SCALE"), "TOR_F_SCALE"
FROM tornadoes2018 
WHERE "BEGIN_YEARMONTH" >= 200701
GROUP BY "TOR_F_SCALE"
ORDER BY count DESC
"""
pd.read_sql(sql_query, Tornado_local)

,count,TOR_F_SCALE
0,8617,EF0
1,5474,EF1
2,1584,EF2
3,445,EF3
4,116,EFU
5,103,EF4
6,14,EF5
7,11,F1
8,8,F0
9,3,F2


In [13]:
sql_query = """
SELECT "BEGIN_YEARMONTH", COUNT("BEGIN_YEARMONTH") as "COUNTTORNADOES"
FROM tornadoes2018 
WHERE "BEGIN_YEARMONTH" >= 200702
GROUP BY "BEGIN_YEARMONTH"
ORDER BY "BEGIN_YEARMONTH" ASC
LIMIT 3
"""
pd.read_sql(sql_query, Tornado_local)

,BEGIN_YEARMONTH,COUNTTORNADOES
0,200702,70
1,200703,208
2,200704,188


In [15]:
# Saving new dataset):
sql_query = """
SELECT * 
FROM tornadoes2018 
WHERE "BEGIN_YEARMONTH" >= 200702;
"""

Tornado2018 = pd.read_sql(sql_query, Tornado_local)

# Counting the number of tornadoes in each magnitude category:
Tornado2018.TOR_F_SCALE.value_counts()

EF0    8617
EF1    5474
EF2    1584
EF3     445
EFU     116
EF4     103
EF5      14
Name: TOR_F_SCALE, dtype: int64

In [40]:
tornado = Tornado2018.copy()
tornado.columns

Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE'],
      dtype='object')

In [41]:
Tornado2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16353 entries, 0 to 16352
Data columns (total 51 columns):
BEGIN_YEARMONTH       16353 non-null int64
BEGIN_DAY             16353 non-null int64
BEGIN_TIME            16353 non-null int64
END_YEARMONTH         16353 non-null int64
END_DAY               16353 non-null int64
END_TIME              16353 non-null int64
EPISODE_ID            16353 non-null float64
EVENT_ID              16353 non-null int64
STATE                 16353 non-null object
STATE_FIPS            16353 non-null float64
YEAR                  16353 non-null int64
MONTH_NAME            16353 non-null object
EVENT_TYPE            16353 non-null object
CZ_TYPE               16353 non-null object
CZ_FIPS               16353 non-null int64
CZ_NAME               16353 non-null object
WFO                   16353 non-null object
BEGIN_DATE_TIME       16353 non-null object
CZ_TIMEZONE           16353 non-null object
END_DATE_TIME         16353 non-null object
INJURIES_DIRECT   

In [42]:
compassdir2degrees = {'N': 0., 'NNE': 22.5, 'NE': 45., 'ENE': 67.5, 'E': 90., 'ESE': 112.5, 
                      'SE': 135., 'SSE': 157.5, 'S': 180., 'SSW': 202.5, 'SW': 225., 
                      'WSW': 247.5, 'W': 270., 'WNW': 292.5, 'NW': 315., 'NNW': 337.5}


def compute_Az_AvLat_AvLon_fromLatLong(df, xA='BEGIN_LON', yA='BEGIN_LAT', xB='END_LON',
                                       yB='END_LAT', outMeanLat='Mean_Lat', 
                                       outMeanLon='Mean_Lon', outAz='Azimuth'):
    LAT = (df[yA]+df[yB])/2
    LON = (df[xA]+df[xB])/2
    df[outMeanLat] = np.round(LAT,4)
    df[outMeanLon] = np.round(LON,4)

    #radius in miles:
    R = 3959
     # Coordinates to radians:
    diffLON = np.radians(df[xB] - df[xA])
    diffLAT = np.radians(df[yB] - df[yA])
      
    # Azimuth computed, help from https://gis.stackexchange.com/questions/81124/formula-for-coordinates-lat-lon-azimuth-and-distance
    AZ = pd.Series(index=diffLON.index)
    for i in range(len(diffLON)):
        if diffLAT[i]==0 and diffLON[i]==0:
            AZ[i] = np.nan
        else:
            AZ[i] = np.degrees(np.arctan2(diffLON[i],diffLAT[i]))

    df[outAz] = np.round(AZ,2)

    #Setting north as default
    df[outAz] = df[outAz].map(lambda x: x-360 if x>180 else x)

    return df

In [43]:
#converting to Integers 
def convertEFtonb(x):
    if x=='EF0' or x=='EFU': return 0
    elif x=='EF1': return 1
    elif x=='EF2': return 2
    elif x=='EF3': return 3
    elif x=='EF4': return 4
    elif x=='EF5': return 5
    else: return np.nan

In [44]:
#converting DF input to datetime format & avg
def computeduration(df, beginDT='BEGIN_DATE_TIME', endDT='END_DATE_TIME'):

    # Changing to datetime format:
    df[beginDT] = pd.to_datetime(df[beginDT])
    df[endDT] = pd.to_datetime(df[endDT])
    # Computes the duration:
    df['Duration'] =  df[endDT] - df[beginDT]
    # Converting to minutes from seconds:
    df['Duration'] = df.Duration.map(lambda x: int(round(x.total_seconds()/60.)))
     # Computing the average date and time of a tornado event:
    df['AverageDate'] = pd.to_datetime(df[beginDT] + (df[endDT] - df[beginDT])/2)
    df['AverageTime'] = df['AverageDate']
    
    def dayoftheyear(x):
        dayYear = x.timetuple().tm_yday
        if calendar.isleap(x.year) and dayYear<=59:
            return dayYear
        #forleapyear
        elif calendar.isleap(x.year) and dayYear>59:
            return dayYear - 1
        # ForNonleap years:
        else:
            return dayYear
        
    # Converting date to day of the year:
    df['AverageDate'] = df['AverageDate'].map(dayoftheyear)
    # Converting time to float:
    df['AverageTime'] = df['AverageTime'].map(lambda x: x.time())
    df['AverageTime'] = df['AverageTime'].map(lambda x: x.hour) \
    + df['AverageTime'].map(lambda x: round((x.minute*100/60.)/100., 2)) \
    + df['AverageTime'].map(lambda x: round((x.second*100/60.)/10000., 4))
    
    return df

In [45]:
testdf = Tornado2018[['BEGIN_DATE_TIME', 'END_DATE_TIME']].head()

computeduration(testdf)

,BEGIN_DATE_TIME,END_DATE_TIME,Duration,AverageDate,AverageTime
0,2007-05-06 15:40:00,2007-05-06 15:43:00,3,126,15.685
1,2007-07-02 17:30:00,2007-07-02 17:35:00,5,183,17.535
2,2007-04-21 18:48:00,2007-04-21 18:57:00,9,111,18.875
3,2007-06-01 11:55:00,2007-06-01 11:56:00,1,152,11.925
4,2007-04-15 06:15:00,2007-04-15 06:15:00,0,105,6.250


In [46]:
def convertcost_tointeger(cost):
    '''From a cost in text format in format 'nb'.00'letter' where letter = K or M or B
    Returns a cost as integer'''
    if pd.isnull(cost): return 0
    elif 'K' in cost: return int(round(float(cost[:-2]) * 10**3))
    elif 'M' in cost: return int(round(float(cost[:-2]) * 10**6))
    elif 'B' in cost: return int(round(float(cost[:-2]) * 10**9))
    else: return np.nan

In [47]:
#initial test
testdf2 = Tornado2018[['DAMAGE_PROPERTY', 'DAMAGE_CROPS']][67:74]
testdf2['PropertyDamageCost'] = testdf2.DAMAGE_PROPERTY.map(convertcost_tointeger)
testdf2['CropsDamageCost'] = testdf2.DAMAGE_CROPS.map(convertcost_tointeger)
testdf2

,DAMAGE_PROPERTY,DAMAGE_CROPS,PropertyDamageCost,CropsDamageCost
67,1.50M,0.00K,1500000,0
68,100.00K,0.00K,100000,0
69,3.50M,None,3500000,0
70,20.00K,0.00K,20000,0
71,0.00K,0.00K,0,0
72,0.00K,0.00K,0,0
73,8.00K,0.00K,8000,0


In [48]:
# webscrape state id's from FIPS:
link = "https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code"
response = requests.get(link)
html = response.text
# Creating a list
soup = BeautifulSoup(html, "lxml")
listofFIPSrows = soup.findAll("tr")
    
fips_to_state = {}
for element in listofFIPSrows[1:75]:
    if str(element).split('<td>')[2][:2] != '</':
        fips_to_state[int(str(element).split('<td>')[3][:2])] = str(element).split('<td>')[2][:2]
#fips_to_state[99] = 'PR'

print("FIPS number and state abreviation dictionary:\n"), fips_to_state

FIPS number and state abreviation dictionary:



(None,
 {1: 'AL',
  2: 'AK',
  60: 'AS',
  4: 'AZ',
  5: 'AR',
  6: 'CA',
  8: 'CO',
  9: 'CT',
  10: 'DE',
  11: 'DC',
  12: 'FL',
  64: 'FM',
  13: 'GA',
  66: 'GU',
  15: 'HI',
  16: 'ID',
  17: 'IL',
  18: 'IN',
  19: 'IA',
  20: 'KS',
  21: 'KY',
  22: 'LA',
  23: 'ME',
  68: 'MH',
  24: 'MD',
  25: 'MA',
  26: 'MI',
  27: 'MN',
  28: 'MS',
  29: 'MO',
  30: 'MT',
  31: 'NE',
  32: 'NV',
  33: 'NH',
  34: 'NJ',
  35: 'NM',
  36: 'NY',
  37: 'NC',
  38: 'ND',
  69: 'MP',
  39: 'OH',
  40: 'OK',
  41: 'OR',
  70: 'PW',
  42: 'PA',
  72: 'PR',
  44: 'RI',
  45: 'SC',
  46: 'SD',
  47: 'TN',
  48: 'TX',
  74: 'UM',
  49: 'UT',
  50: 'VT',
  51: 'VA',
  78: 'VI',
  53: 'WA',
  54: 'WV',
  55: 'WI',
  56: 'WY'})

In [49]:
#Testing again
testdf3 = Tornado2018[['STATE_FIPS','STATE']][784:790]
testdf3['State'] = testdf3['STATE_FIPS'].map(lambda x: fips_to_state[x])
testdf3

,STATE_FIPS,STATE,State
784,31.0,NEBRASKA,NE
785,20.0,KANSAS,KS
786,20.0,KANSAS,KS
787,48.0,TEXAS,TX
788,31.0,NEBRASKA,NE
789,20.0,KANSAS,KS


In [50]:
# #Confusion matrix plot setup, ended up not doing...not enought time, but great idea for next time. 
# def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues, 
#                           xlab='Predicted magnitude', ylab='True magnitude'):
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)
#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, cm[i, j],
#                  horizontalalignment="center",
#                  color="white" if cm[i, j] > thresh else "black")
#     plt.ylabel(ylab)
#     plt.xlabel(xlab)
#     plt.show()

In [51]:
#dropping columns 
tornado = tornado.drop(['EVENT_ID', 'EPISODE_ID', 
                                            'EVENT_TYPE', 'MAGNITUDE', 'MAGNITUDE_TYPE', 
                                            'FLOOD_CAUSE', 
                                            'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 
                                            'DATA_SOURCE', 'CATEGORY'], axis=1)

# Adding new columns, duration,average year-date, and time: 
tornado = computeduration(tornado)
# removing original time columns
tornado.drop(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH', 'END_DAY',
                        'END_TIME', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME', 'YEAR',
                        'MONTH_NAME'], axis=1, inplace=True)

# direct&indirect deaths/injuries
tornado['Deaths'] = tornado.DEATHS_DIRECT + tornado.DEATHS_INDIRECT
tornado['Injuries'] = tornado.INJURIES_DIRECT +\
                                tornado.INJURIES_INDIRECT
tornado.drop(['DEATHS_DIRECT','DEATHS_INDIRECT','INJURIES_DIRECT','INJURIES_INDIRECT'],
                       axis=1, inplace=True)

# Adds 3 new columns average latitude, longitude and azimuth of tornado's path:
tornado = compute_Az_AvLat_AvLon_fromLatLong(tornado)

In [54]:
#export to CSV 
tornado.to_csv(r'Resources/CleanTornado.csv')